In [0]:
from pyspark.sql.functions import col, unix_timestamp, to_date, dayofweek
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read.table("workspace.default.bronze_taxi").filter(
    col("tpep_pickup_datetime").isNotNull() & col("tpep_dropoff_datetime").isNotNull()
)

df = df.withColumn("trip_duration_seconds", unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) \
       .withColumn("trip_date", to_date("tpep_pickup_datetime")) \
       .withColumn("pickup_day_of_week", dayofweek("tpep_pickup_datetime"))

df.write.mode("overwrite").format("delta").saveAsTable("silver_taxi")
